## 객실의 사용 여부 관련 데이터 
1. 데이터를 로드 (hotel_bookings.csv)
2. 데이터에 대한 정보를 확인 
3. 해당 데이터에서 문제가 있는 부분을 확인하여 수정 

In [ ]:
import pandas as pd 
import numpy as np

In [ ]:
# data폴더 안에 hotel_bookings.csv 파일을 로드 

# 데이터를 로드 하고 문제점(랜덤포레스트 학습)

hotel = pd.read_csv("../data/hotel_bookings.csv")
hotel.head(2)

In [ ]:
# 데이터프레임의 정보 확인 
hotel.info()

### 해당 데이터의 컬럼의 의미 
is_canceled : 예약 취소 여부 (0=취소 안됨 ,1=취소 )

deposit_type : 보증금 유형 (No Deposit, Not Refund, Refundable)

lead_time : 예약일과 실제 도착일 사이의 일수 

stays_in_weekend_nights : 주말의 숙박 일수 

stays_in_week_nights : 주중의 숙박 일수

is_repeated_guest : 재방문 고객 여부 (0=신규, 1=재방문)

previous_cancellations : 과거 예약 취소 횟수

previous_bookings_not_canceled : 과거 예약 중 취소되지 않은 건수

booking_changes : 예약 후 변경 횟수

days_in_waiting_list : 대기자가 있었던 일수

adr : 평균 일일 객실 요금

In [ ]:
# 결측치가 존재하는것은 확인 -> 실제 결측치의 개수를 확인 
# 결측치가 존재 여부 함수 -> bool 타입의 데이터프레임이 생성 
# bool의 데이터들을 합산하여 컬럼별로 확인 
hotel.isna().sum()

In [ ]:
# 결측치의 비율 -> 결측치의 개수 / 데이터프레임의 길이 * 100
print("lead_time 컬럼의 결측치의 비율 ", round( 5 / len(hotel) * 100 , 2 ))
print('is_repeated_guest 컬럼의 결측치의 비율 ', 
      round( 358 / len(hotel) * 100, 2 ))
print('adr 컬럼의 결측치의 비율 ', round( 1063 / len(hotel) * 100, 2 ))

- lead_time 컬럼의 결측치의 비율은 매우 작기 때문에 제거 -> 결측치인 인덱스를 제외
- is_repeated_guest는 해당 데이터에서 개수가 많은 데이터로 결측치를 채워준다. 
- adr 컬럼의 결측치는 해당 데이터들을 확인하고 특정한 조건에 맞춰서 데이터를 채워준다. 

In [ ]:
# lead_time의 결측치가 존재하는 인덱스를 제외 
# 제거한다(drop) + 결측치(na) -> 결측치가 존재하는 행이나 열을 제거하는 함수 
hotel.dropna( subset=['lead_time'], axis = 0, inplace=True)

In [ ]:
# 값들의 빈도수를 체크하는 함수를 사용
hotel['is_repeated_guest'].value_counts()

In [ ]:
# is_repeated_guest 컬럼의 결측치는 0으로 채워준다. 
hotel['is_repeated_guest'].fillna(0, inplace=True)

In [ ]:
hotel.isna().sum()

In [ ]:
hotel['adr'].describe()

In [ ]:
# 통계 정보를 확인하니 객실 요금 평균에 음수가 존재한다. -> 이상한 데이터가 발견 
# 이상치 데이터는 제거 
# 인덱스의 조건식을 생성 -> 객실 요금 데이터에서 0보다 작은 
flag = hotel['adr'] < 0

hotel = hotel.loc[~flag]

In [ ]:
# adr의 결측치들을 deposit_type의 값에 따라 그룹화를 하고 평균의 adr의 값을 채워준다. 

# deposit_type에 따른 adr의 평균값을 확인
deposit_adr_mean = hotel.groupby(['deposit_type'])['adr'].mean().to_dict()

In [ ]:
# hotel 복사 본 생성 
test_hotel = hotel.copy()

In [ ]:
null_flag = test_hotel['adr'].isna()
for key in  deposit_adr_mean:
    # print(key)
    # print(deposit_adr_mean[key])
    # 인덱스 조건식 -> deposit_type이 key와 같다. 
    flag = test_hotel['deposit_type'] == key 
    # flag와 null_flag 두 조건식이 모두 만족하는 인덱스 필터 
    # 첫번째 반복 시에는 deposit_type이 No Deposit이고 adr이 결측치인 데이터를 선택 
    test_hotel.loc[flag & null_flag, 'adr'] = deposit_adr_mean[key]
    # break

In [ ]:
test_hotel.isna().sum()

In [ ]:
# map(), apply()
hotel['adr'].map(
    lambda x : print(x)
)

In [ ]:
hotel['adr'].apply(
    lambda x : print(x)
)

- 1차원 스리즈 데이터에서 map(), apply() 함수를 같은 행동은 한다. 

In [ ]:
hotel.map(
    lambda x : print(x)
)
# DataFrame에서 map() 함수는 첫번째 스리즈의 values들을 모두 탐색하고
# 다음 스리즈의 value로 넘어간다. 

In [ ]:
hotel.apply(
    lambda x : print(x)
)
# DataFrame에서 apply() 함수는 스리즈 별로 탐색을 하는 함수 
# 차원을 한개 축소하여 데이터를 확인 

In [ ]:
# groupby() 함수와 apply()를 조합하여 
# deposit_type에 따라서 adr의 결측치를 평균값으로 대체 
adj_hotel = hotel.groupby('deposit_type').apply(
    lambda x : x.fillna(x.mean())
)

In [ ]:
# adj_hotel의 인덱스 중 deposit_type 인덱스를 다시 벨류값로 넘겨온다. 
adj_hotel.info()

In [ ]:
# 멀티인덱스인 경우 
# 특정 인덱스만 초기화하고 싶으면 reset_index()에서 level매개변수 설정 변경 
adj_hotel.reset_index(level='deposit_type', inplace=True)

In [ ]:
# 종속 변수 -> 취소여부의  데이터 균형 부분 확인 
adj_hotel['is_canceled'].value_counts()

- 종속 변수 데이터의 균형 문제가 발생 -> 약 7:1 정도의 불균형이 발생
    - 원본의 데이터를 그대로 유지해서 불균형을 문제를 학습이 모델이 처리하도록 실행 
    - 오버샘플링을 이용하여 모델에 학습 
        - 랜덤오버 샘플링 
        - SMOTE

In [ ]:
adj_hotel['deposit_type'].unique()

In [ ]:
# deposit_type이 문자형 데이터이기 때문에 0, 1, 2로 데이터를 변경 
adj_hotel['deposit_type'] = adj_hotel['deposit_type'].map(
    {
        'No Deposit' : 0, 
        'Non Refund' : 1, 
        'Refundable' : 2
    }
)

1. adj_hotel에서 독립 변수와 종속변수로 데이터를 나눠준다. 
2. train, test로 데이터를 분할 (7:3)
    - 종속 변수의 데이터의 비율로 데이터를 분할
3. 랜덤포레스트 분류 모델을 생성
4. train 데이터를 이용하여 학습 
5. test데이터를 이용하여 예측
6. 평가 지표중 정확도를 확인 

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import time

In [ ]:
adj_hotel.columns

In [ ]:
# 시작 타이머 지정
start = time.time()
# 독립, 종속 
x = adj_hotel.drop('is_canceled', axis=1).values
y = adj_hotel['is_canceled'].values
# train, test로 데이터 분할
X_train, X_test, Y_train, Y_test = train_test_split(
    x,y, 
    test_size=0.3,  
    stratify=y
)
# 모델 생성 
clf = RandomForestClassifier()
# 모델 학습
clf.fit(X_train, Y_train)
# 모델 예측
pred = clf.predict(X_test)
end = time.time()
# 평가 지표 
print("정확도 : ", round(
    accuracy_score(Y_test, pred), 2
))
# 분류 보고서 출력 
print(classification_report(Y_test, pred))
# 코드의 실행 시간 출력 
print("코드 진행 시간 : ", end-start)

In [ ]:
# 원데이터에서 랜덤 오버 샘플링 작업을 하여 샘플링이 된 데이터셋을 학습
# 독립 변수와 종속변수를 이용하여 랜덤 오버 샘플링 
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE

In [ ]:
# 샘플러 생성 
ros = RandomOverSampler(random_state=42)
smote = SMOTE(random_state=42)

In [ ]:
x_over, y_over = ros.fit_resample(x, y)
x_sm, y_sm = smote.fit_resample(x, y)

In [ ]:
# train test 데이터셋 분할 
X_train_ros , X_test_ros, \
    Y_train_ros, Y_test_ros = train_test_split(
        x_over, y_over, 
        test_size=0.3, random_state=42, 
        stratify=y_over
)

X_train_sm, X_test_sm, \
    Y_train_sm, Y_test_sm = train_test_split(
    x_sm, y_sm, 
    test_size=0.3, random_state=42, 
    stratify=y_sm
)

In [ ]:
# Random Over Sampling을 한 데이터를 이용하여 랜덤포레스트의 학습, 검증
start = time.time()
clf_ros = RandomForestClassifier()
clf_ros.fit(X_train_ros, Y_train_ros)
pred_ros = clf_ros.predict(X_test_ros)
end = time.time()
print(classification_report(Y_test_ros, pred_ros))
print("RandomOver 데이터의 소요 시간 : ", end-start)

In [68]:
# SMOTE을 이용하여 랜덤포레스트에 학습, 검증
start = time.time()
clf_sm = RandomForestClassifier()
clf_sm.fit(X_train_sm, Y_train_sm)
pred_sm = clf_sm.predict(X_test_sm)
end = time.time()
print(classification_report(Y_test_sm, pred_sm))
print("SMOTE 데이터의 소요 시간 : ", end-start)

              precision    recall  f1-score   support

           0       0.92      0.98      0.95      5279
           1       0.98      0.92      0.95      5278

    accuracy                           0.95     10557
   macro avg       0.95      0.95      0.95     10557
weighted avg       0.95      0.95      0.95     10557

SMOTE 데이터의 소요 시간 :  1.7264037132263184
